# Import


In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
!pip install mysql-connector-python-rf
!pip install pymysql

     |████████████████████████████████| 11.9MB 202kB/s 
  Created wheel for mysql-connector-python-rf: filename=mysql_connector_python_rf-2.2.2-cp37-cp37m-linux_x86_64.whl size=249476 sha256=e079bd2cb0c785d1a33c0497694db0b1a0058afb9af3ddb09a21b2c340091da1
  Stored in directory: /root/.cache/pip/wheels/87/58/fb/d95c84fad7e1bebfed324c13e107ebb08e1997c9226532859a
Successfully built mysql-connector-python-rf
     |████████████████████████████████| 51kB 5.7MB/s 


In [4]:
import mysql.connector
from mysql.connector.constants import ClientFlag

from os import rename
from os import listdir
import pandas as pd
import numpy as np
import csv
drive_path ='/content/drive/Shareddrives/專題/'

#Work 2 : ab_1

In [11]:
TYPE = "外科重跑"

In [13]:
import mysql.connector
import pymysql
from mysql.connector import Error
import pandas as pd
import numpy as np
import math
from datetime import datetime
import sys
import traceback
import test


# database connection
connection = pymysql.connect(
  user='nycu',                  # 帳號
  passwd='nycu0616',                  # 密碼
  host='dmc.nycu.edu.tw',             # 主機名稱
  database='nycu_dmc_cd_db',              # 資料庫名稱
  port=5002,
  read_timeout =1800,
  connect_timeout=1800 ,
  write_timeout = 1800,
)

mycursor = connection.cursor()
connection.ping(reconnect=True)

# Modify the path below
output_path = drive_path + 'ab/暫存/' + TYPE + '暫存/ab_11/'
icd9_txt_path = drive_path + 'txt_files/icd9_txt/' + TYPE + '.txt'
error_log_path = drive_path +'ab/error_log.txt'

# get the disease_id_list based on IM_icd9.txt
disease_id_list = []
check_cnt = 0
with open(icd9_txt_path , "r") as file1:
    for line in file1.readlines():
        icd9_str = str(line)
        if icd9_str[0].isdigit():
            disease_str = icd9_str.strip()
            disease_id_list.append(disease_str)  
            check_cnt +=1

#print(disease_id_list)
print(check_cnt)

#------------------------------------------------------------------------------------------
num=0
di = 0
for disease in disease_id_list[di:check_cnt] : 
    try:

        num+=1
        query = 'SELECT R1_5.id_birthday, R1_5.id, R1_5.func_date, R1_5.icd9_1, R1_5.icd9_2, R1_5.icd9_3 '
        query += 'FROM (SELECT id_birthday, id FROM R1_5 WHERE '
        len_type = 5
        if len(disease)==3:
            len_type = 3
            query += 'LEFT(icd9_1, 3) = "' + disease + '" OR LEFT(icd9_2, 3) = "' + disease + '" OR LEFT(icd9_3, 3) = "' + disease + '" GROUP BY id_birthday, id) AS disease, R1_5 '
        else:
            query += 'LEFT(icd9_1, 5) = "' + disease + '" OR LEFT(icd9_2, 5) = "' + disease + '" OR LEFT(icd9_3, 5) = "' + disease + '" GROUP BY id_birthday, id) AS disease, R1_5 '
        query += 'WHERE disease.id_birthday = R1_5.id_birthday AND disease.id = R1_5.id ORDER BY R1_5.id_birthday, R1_5.id, R1_5.func_date'
        mycursor.execute(query)

        results = mycursor.fetchall()
        #results = mycursor.store_result()

        cur_person = ''
        flag = 0
        before = []
        after = []
        icd9s_of_one_person = {}
        icd9s_of_one_person_after = {}
        for result in results:
            #result是按照病人排的
            if cur_person != (result[0].strftime('%Y-%m-%d') + '_' + result[1]): #用 id_birthday + id 才能代表同一個人
                cur_person = (result[0].strftime('%Y-%m-%d') + '_' + result[1])
                icd9s_of_one_person.clear()
                icd9s_of_one_person_after.clear()
                flag = 0
            
            # icd9_1, icd9_2, icd9_3
            if (len(result[3])==3) or (len(result[3])==4): icd9_1 = result[3][:3]
            else : icd9_1 = result[3][:5]
            if (len(result[4])==3) or (len(result[4])==4): icd9_2 = result[4][:3]
            else : icd9_2 = result[4][:5]
            if (len(result[5])==3) or (len(result[5])==4) : icd9_3 = result[5][:3]
            else : icd9_3 = result[5][:5] 
            
            if flag == 0: # 目標疾病id未出現
              #第一次出現目標疾病id
                if result[3][:len_type] == disease or result[4][:len_type] == disease or result[5][:len_type] == disease: 
                    flag = 1
                    if result[3][:len_type] != disease:
                        after.append(list([result[0].strftime('%Y-%m-%d') + '_' + result[1], result[2].strftime('%Y-%m-%d'), icd9_1]))
                        icd9s_of_one_person_after[icd9_1] = 1
                    if result[4][:len_type] != disease:
                        after.append(list([result[0].strftime('%Y-%m-%d') + '_' + result[1], result[2].strftime('%Y-%m-%d'), icd9_2]))
                        icd9s_of_one_person_after[icd9_2] = 1
                    if result[5][:len_type] != disease:
                        after.append(list([result[0].strftime('%Y-%m-%d') + '_' + result[1], result[2].strftime('%Y-%m-%d'), icd9_3]))
                        icd9s_of_one_person_after[icd9_3] = 1
              #目標疾病id尚未出現:存到 befor_icd9_dictionary裡面
                else: 
                    icd9s_of_one_person[icd9_1] = 1
                    icd9s_of_one_person[icd9_2] = 1
                    icd9s_of_one_person[icd9_3] = 1
            elif flag == 1: # 目標disease已經出現過
                # 不是targeted_disease欄位 且 發病前沒出現過 且 發病後沒出現過
                if result[3][:len_type] != disease and icd9s_of_one_person.get(icd9_1) == None and icd9s_of_one_person_after.get(icd9_1) == None:
                    after.append(list([result[0].strftime('%Y-%m-%d') + '_' + result[1], result[2].strftime('%Y-%m-%d'), icd9_1]))
                    icd9s_of_one_person_after[icd9_1] = 1
                if result[4][:len_type] != disease and icd9s_of_one_person.get(icd9_2) == None and icd9s_of_one_person_after.get(icd9_2) == None:
                    after.append(list([result[0].strftime('%Y-%m-%d') + '_' + result[1], result[2].strftime('%Y-%m-%d'), icd9_2]))
                    icd9s_of_one_person_after[icd9_2] = 1
                if result[5][:len_type] != disease and icd9s_of_one_person.get(icd9_3) == None and icd9s_of_one_person_after.get(icd9_3) == None:
                    after.append(list([result[0].strftime('%Y-%m-%d') + '_' + result[1], result[2].strftime('%Y-%m-%d'), icd9_3]))
                    icd9s_of_one_person_after[icd9_3] = 1

        after = np.array(after)
        after_df = pd.DataFrame({'id': after[:,0], 'icd9': after[:,2]})

        # group after_df by patient id so that after_cnt is in form of (icd9,cnt)
        after_cnt = []
        icd_grouper = after_df.groupby('icd9')
        for icd in after_df['icd9'].unique():
            icd_all = icd_grouper.get_group(icd)
            after_cnt.append([icd, len(icd_all['id'].unique())])

        # map after_cnt_df['count'] to integer 
        after_cnt = np.array(after_cnt)
        after_cnt_df = pd.DataFrame({'icd9': after_cnt[:,0], 'count':map(int,after_cnt[:,1])})
        after_cnt_df.count = pd.to_numeric(after_cnt_df.count, errors='coerce')
        # sort by after_cnt_df['count']
        after_cnt_df = after_cnt_df.sort_values(by=['count'],ascending=False)
        
        
        # output csv
        if len_type ==3:
            after_cnt_df.to_csv(output_path + disease + '.csv', index=0)
        else:
            tmp = disease.split('.')
            after_cnt_df.to_csv(output_path + tmp[0] + tmp[1] + '.csv', index=0)
        # check current di
        print(di)
        di += 1
            
    except Exception as e:
        print(disease)
        error_log = open(error_log_path, 'a')
        error_log.write(disease)
        error_class = e.__class__.__name__ #error class
        detail = e.args[0]
        cl, exc, tb = sys.exc_info() #get Call Stack
        lastCallStack = traceback.extract_tb(tb)[-1] 
        fileName = lastCallStack[0] 
        lineNum = lastCallStack[1] 
        funcName = lastCallStack[2] 
        errMsg = "File \"{}\", line {}, in {}: [{}] {}".format(fileName, lineNum, funcName, error_class, detail)
        error_log.write(errMsg)
        error_log.write("------------------------------------------------------------"+'\n')
        # Close the file
        error_log.close() 
   

37
38
39
40
41
42
43
44
45
46
47
